## Data Fetching
This interactive notebook handles the fetching and cleaning of data from the EPA's Air Quality System. To do so, we should start by importing some of our required libraries, including our custom created `pyaqs` module that provides wrapper methods to convert information stored in the AQS REST API to easily accessible and modifiable Pandas dataframes.

In [28]:
from pyaqs import AQSFetcher
import pandas as pd
from time import sleep
import numpy as np

Now, we will instantiate a new AQSFetcher object and use it to get the required data from the EPA website. From our previous investigation of the four states of Illinois, New York, California, and Georgia, we have concluded that we need more data in order to draw any substantial conclusions, negative or positive. Therefore, in this notebook, we will be fetching data for all 50 states, for the criteria parameters as defined by the EPA.

To note, in this context, a *parameter* is a compound that in the air that can be measured. The EPA has many such parameters, sorted into different classes whose descriptions are easily accessible through the API.

Let's start by instantiating a fetcher object, and we'll use it to get the list of state codes so that we can easily query the API in an automated way.

In [8]:
aqs_fetcher = AQSFetcher('bbjornstad.flatiron@gmail.com', 'ochrefox21')

In [9]:
state_codes = aqs_fetcher.get_state_codes()
state_codes.head()

,code,state_name
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


Next, let's take a look at the possible parameter classes and identify a set that seems reasonable for analysis.

In [10]:
aqs_fetcher.get_parameter_classes()

,class_name,class_description
0,AIRNOW MAPS,The parameters represented on AirNow maps (881...
1,ALL,Select all Parameters Available
2,AQI POLLUTANTS,Pollutants that have an AQI Defined
3,CORE_HAPS,Urban Air Toxic Pollutants
4,CRITERIA,Criteria Pollutants
5,CSN DART,List of CSN speciation parameters to populate ...
6,FORECAST,Parameters routinely extracted by AirNow (STI)
7,HAPS,Hazardous Air Pollutants
8,IMPROVE CARBON,IMPROVE Carbon Parameters
9,IMPROVE_SPECIATION,PM2.5 Speciated Parameters Measured at IMPROVE...


We are most interested in those parameters held in the CRITERIA class, as indicated by the description. In particular, this class defines pollutants that the EPA has determined to be suitable criteria for overall air quality. Let's store these parameters and their associated codes for easy grouping later on.

In [11]:
parameter_codes = aqs_fetcher.get_parameter_list_by_class('CRITERIA')
parameter_codes

,code,parameter_description
0,14129,Lead (TSP) LC
1,42101,Carbon monoxide
2,42401,Sulfur dioxide
3,42602,Nitrogen dioxide (NO2)
4,44201,Ozone
5,81102,PM10 Total 0-10um STP
6,85129,Lead PM10 LC FRM/FEM
7,88101,PM2.5 - Local Conditions


Fantastic, these will allow us the possibility to easily partition and query the data that we need to continue with the analysis. Let's now use these parameters and our AQSFetcher object to get annual summary data by state between the periods of 2012 to 2016.

*WARNING: this cell will take quite some time to run, due to the nature of fetching data from a web API. This raw dataframe is saved to the [`data`](./data/) folder, to save quite a bit of time and further headache*

In [12]:
all_state_aq_data = pd.DataFrame()

for state_code in state_codes.code:
    all_state_aq_data = pd.concat(
        [all_state_aq_data, 
         aqs_fetcher.annual_data_by_state(state_code, parameter_codes.code, 20120101, 20161231)])
    # sleep to not break the API
    sleep(np.random.rand()*0.25)

all_state_aq_data.head()

No matching data could be found!
Bad URL!


,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration,...,fiftieth_percentile,tenth_percentile,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,01,083,0005,14129,1,34.645848,-86.947311,WGS84,Lead (TSP) LC,24 HOUR,...,0.004,0.002,Pryor Field Lead Monitoring Site,"US 31, Local On Calhoun Community College",Alabama,Limestone,Athens,26620,"Huntsville, AL",2013-06-28
1,01,073,2059,42101,1,33.521427,-86.844112,WGS84,Carbon monoxide,1 HOUR,...,0.370,0.240,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2017-04-18
2,01,073,2059,42101,1,33.521427,-86.844112,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,0.400,0.300,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2017-04-18
3,01,073,2059,42101,1,33.521427,-86.844112,WGS84,Carbon monoxide,1 HOUR,...,0.380,0.260,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2017-04-18
4,01,073,2059,42101,1,33.521427,-86.844112,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,0.400,0.300,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,13820,"Birmingham-Hoover, AL",2017-04-18


In [13]:
all_state_aq_data.to_csv('data/all_states_raw_aq.csv')

In [33]:
# uncomment to read in the raw dataframe if we don't want to go through the process of fetching again
# all_state_aq_data = pd.read_csv('data/all_states_raw_aq.csv', index_col=0)

all_state_aq_data.shape

(90480, 55)

Let's also do some paring of this large number of columns. Many of these fields are superfluous for our analysis and so we can simply drop the columns (or in this case, keep the columns that we want).

In [34]:
all_state_aq_data.columns

Index(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter', 'sample_duration',
       'pollutant_standard', 'metric_used', 'method', 'year',
       'units_of_measure', 'event_type', 'observation_count',
       'observation_percent', 'validity_indicator', 'valid_day_count',
       'required_day_count', 'exceptional_data_count',
       'null_observation_count', 'primary_exceedance_count',
       'secondary_exceedance_count', 'certification_indicator',
       'arithmetic_mean', 'standard_deviation', 'first_max_value',
       'first_max_datetime', 'second_max_value', 'second_max_datetime',
       'third_max_value', 'third_max_datetime', 'fourth_max_value',
       'fourth_max_datetime', 'first_max_nonoverlap_value',
       'first_max_n_o_datetime', 'second_max_nonoverlap_value',
       'second_max_n_o_datetime', 'ninety_ninth_percentile',
       'ninety_eighth_percentile', 'ninety_fifth_percentile',
       'ninetieth_perc

In [35]:
cols_to_drop = ['state_code', 'poc', 'latitude', 'longitude', 'datum', 'event_type', 'observation_percent', 'validity_indicator',
                'valid_day_count', 'required_day_count', 'primary_exceedance_count', 'secondary_exceedance_count', 
                'certification_indicator', 'first_max_value', 'first_max_datetime', 'second_max_value', 'second_max_datetime',
                'third_max_value', 'third_max_datetime', 'fourth_max_value', 'fourth_max_datetime', 'first_max_nonoverlap_value',
                'first_max_n_o_datetime', 'second_max_nonoverlap_value', 'second_max_n_o_datetime', 'ninety_ninth_percentile',
                'ninety_eighth_percentile', 'ninety_fifth_percentile', 'ninetieth_percentile', 'seventy_fifth_percentile',
                'fiftieth_percentile', 'tenth_percentile', 'cbsa_code', 'cbsa', 'pollutant_standard', 'method', 'metric_used']
all_state_aq_data.drop(columns=cols_to_drop, inplace=True)

In [36]:
all_state_aq_data.head()

,county_code,site_number,parameter_code,parameter,sample_duration,year,units_of_measure,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation,local_site_name,site_address,state,county,city,date_of_last_change
0,83,5,14129,Lead (TSP) LC,24 HOUR,2012,Micrograms/cubic meter (LC),58,0,4,0.006362,0.006212,Pryor Field Lead Monitoring Site,"US 31, Local On Calhoun Community College",Alabama,Limestone,Athens,2013-06-28
1,73,2059,42101,Carbon monoxide,1 HOUR,2014,Parts per million,8106,0,654,0.432750,0.216344,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,2017-04-18
2,73,2059,42101,Carbon monoxide,8-HR RUN AVG END HOUR,2014,Parts per million,8415,0,0,0.436399,0.176097,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,2017-04-18
3,73,2059,42101,Carbon monoxide,1 HOUR,2015,Parts per million,7188,0,1572,0.440380,0.215847,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,2017-04-18
4,73,2059,42101,Carbon monoxide,8-HR RUN AVG END HOUR,2015,Parts per million,7460,0,0,0.443968,0.178687,Arkadelphia/Near Road,"1110 5th Street West Birmingham, AL 35204",Alabama,Jefferson,Birmingham,2017-04-18


Let's also do a bit of investigation into the consistency of the data. In particular, there are various possible values represented in the `units_of_measure` field, leading us to suspect that perhaps we will need to do a bit of unit conversion before we are ready to begin analysis. To check if this is the case, we can group by the parameter name toogether with the units of measure. If we see that each name is associated to only a single unit, then we will know that we won't have to perform any unit conversions in order to compare within each compound present in the air.

In [37]:
all_state_aq_data.groupby(['parameter', 'units_of_measure']).count()

,,county_code,site_number,parameter_code,sample_duration,year,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation,local_site_name,site_address,state,county,city,date_of_last_change
parameter,units_of_measure,,,,,,,,,,,,,,,,
Carbon monoxide,Parts per million,3314,3314,3314,3314,3314,3314,3314,3314,3314,3314,3098,3314,3314,3314,2996,3314
Lead (TSP) LC,Micrograms/cubic meter (LC),1335,1335,1335,1335,1335,1335,1335,1335,1335,1335,1155,1335,1335,1335,1071,1335
Lead PM10 LC FRM/FEM,Micrograms/cubic meter (LC),181,181,181,181,181,181,181,181,181,181,181,181,181,181,164,181
Nitrogen dioxide (NO2),Parts per billion,4499,4499,4499,4499,4499,4499,4499,4499,4499,4499,4181,4499,4499,4499,3497,4499
Ozone,Parts per million,29525,29525,29525,29525,29525,29525,29525,29525,29523,29523,28093,29525,29525,29525,20049,29525
PM10 Total 0-10um STP,Micrograms/cubic meter (25 C),8303,8303,8303,8303,8303,8303,8303,8303,8303,8303,7802,8303,8303,8303,6701,8303
PM2.5 - Local Conditions,Micrograms/cubic meter (LC),32456,32456,32456,32456,32456,32456,32456,32456,32456,32456,31523,32448,32456,32456,27811,32456
Sulfur dioxide,Parts per billion,10867,10867,10867,10867,10867,10867,10867,10867,10867,10867,9785,10867,10867,10867,8542,10867


We see that in fact, it is the case that the units have already been standardized. Therefore, we don't need to do any unit conversion if we want to gain some actionable insights when making comparisons within a particular parameter. We also see that we have a lot more readings for the Ozone and PM 2.5 parameters.

Let's further clean up the data by aggregating over the full period of time that is available, grouping by county, state, and parameter. This will allow us to gain consistency with the format of the CDC cancer data set that we have chosen to use.

In [38]:
all_state_aggregate_aq = all_state_aq_data.groupby(by=['state', 'county', 'parameter']).mean().reset_index()

In [39]:
all_state_aggregate_aq.head(20)

,state,county,parameter,county_code,site_number,parameter_code,year,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation
0,Alabama,Baldwin,Ozone,3.0,10.000000,44201.0,2014.000000,5176.850000,0.000000,120.200000,0.041641,0.012011
1,Alabama,Baldwin,PM2.5 - Local Conditions,3.0,10.000000,88101.0,2014.000000,116.800000,0.000000,5.200000,8.469895,3.431420
2,Alabama,Clay,PM2.5 - Local Conditions,27.0,1.000000,88101.0,2014.000000,116.200000,0.000000,6.200000,8.291026,3.646928
3,Alabama,Colbert,Ozone,33.0,1002.000000,44201.0,2014.000000,5335.700000,0.000000,76.600000,0.041134,0.010685
4,Alabama,Colbert,PM2.5 - Local Conditions,33.0,1002.000000,88101.0,2014.000000,118.600000,0.000000,3.400000,8.589856,3.836046
5,Alabama,DeKalb,Ozone,49.0,9991.000000,44201.0,2014.000000,7877.450000,0.000000,116.850000,0.044970,0.009932
6,Alabama,DeKalb,PM2.5 - Local Conditions,49.0,1003.000000,88101.0,2014.000000,114.000000,0.000000,7.800000,8.929633,3.813341
7,Alabama,Elmore,Ozone,51.0,1.000000,44201.0,2014.000000,4766.950000,0.000000,75.050000,0.041555,0.010264
8,Alabama,Etowah,Ozone,55.0,11.000000,44201.0,2014.000000,5257.500000,0.000000,97.350000,0.042401,0.010022
9,Alabama,Etowah,PM2.5 - Local Conditions,55.0,10.000000,88101.0,2014.000000,113.800000,0.000000,8.000000,9.099492,3.546380


In [40]:
all_state_aggregate_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 12 columns):
state                     2921 non-null object
county                    2921 non-null object
parameter                 2921 non-null object
county_code               2921 non-null float64
site_number               2921 non-null float64
parameter_code            2921 non-null float64
year                      2921 non-null float64
observation_count         2921 non-null float64
exceptional_data_count    2921 non-null float64
null_observation_count    2921 non-null float64
arithmetic_mean           2921 non-null float64
standard_deviation        2921 non-null float64
dtypes: float64(9), object(3)
memory usage: 274.0+ KB


For ease of merging later on, let's also convert the state column into their standard abbreviations. We'll also remove data points from areas not included in the usual 50 states plus DC, such as readings from Puerto Rico. This is to facilitate ease of merging with the CDC data later on, as cancer rates are not included in that data for these areas. To do so, we'll need a dictionary containing the standard abbreviations associated to state names.

In [27]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

Now we can use this to convert the state column. First, let's remove the extra areas of Mexico, Puerto Rico, and the Virgin Islands.

In [41]:
all_state_aggregate_aq = all_state_aggregate_aq.loc[
    (all_state_aggregate_aq.state != 'Country Of Mexico')
    & (all_state_aggregate_aq.state != 'Virgin Islands')
    & (all_state_aggregate_aq.state != 'Puerto Rico')]

In [42]:
all_state_aggregate_aq.state = all_state_aggregate_aq.state.map(lambda s: us_state_abbrev[s])

In [43]:
all_state_aggregate_aq.head()

,state,county,parameter,county_code,site_number,parameter_code,year,observation_count,exceptional_data_count,null_observation_count,arithmetic_mean,standard_deviation
0,AL,Baldwin,Ozone,3.0,10.0,44201.0,2014.0,5176.85,0.0,120.2,0.041641,0.012011
1,AL,Baldwin,PM2.5 - Local Conditions,3.0,10.0,88101.0,2014.0,116.80,0.0,5.2,8.469895,3.431420
2,AL,Clay,PM2.5 - Local Conditions,27.0,1.0,88101.0,2014.0,116.20,0.0,6.2,8.291026,3.646928
3,AL,Colbert,Ozone,33.0,1002.0,44201.0,2014.0,5335.70,0.0,76.6,0.041134,0.010685
4,AL,Colbert,PM2.5 - Local Conditions,33.0,1002.0,88101.0,2014.0,118.60,0.0,3.4,8.589856,3.836046


Finally, let's save our cleaned and aggregated dataframe as a csv file in the [`cleaned_data`](./cleaned_data/) folder, for ease of access later on.

In [44]:
all_state_aggregate_aq.to_csv('cleaned_data/all_states_aq.csv')